In [ ]:
from os import getenv
from pathlib import Path

input_folder = Path(getenv(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input'))
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output'))
log_folder = Path(getenv(
    'CROSSCOMPUTE_LOG_FOLDER', 'tests/standard/log'))

output_folder.mkdir(parents=True, exist_ok=True)
log_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
GOOGLE_KEY = getenv('GOOGLE_KEY')

In [ ]:
import json

with (input_folder / 'variables.dictionary').open('rt') as f:
    d = json.load(f)
districts_uri = d['districts_uri']
destination_address = d['destination_address']
travel_mode = d['travel_mode']
travel_name = {
    'driving': 'car',
    'transit': 'public transit',
}[travel_mode]

In [ ]:
from hashlib import blake2b

def get_hash(text):
    h = blake2b()
    h.update(text.encode())
    return h.hexdigest()

In [ ]:
from shapely.geometry import mapping, shape

SIMPLIFICATION_TOLERANCE = 0.001

def simplify_feature(feature):
    raw_geometry = shape(feature['geometry'])
    simplified_geometry = raw_geometry.simplify(SIMPLIFICATION_TOLERANCE)
    feature['geometry'] = mapping(simplified_geometry)
    return feature

In [ ]:
import json
from urllib.request import urlretrieve as download_url

datasets_folder = Path('datasets')
districts_path = (
    datasets_folder / 'districts' / get_hash(districts_uri)
).with_suffix('.json')
if not districts_path.exists():
    districts_path.parent.mkdir(parents=True, exist_ok=True)
    raw_path = districts_path.with_suffix('.raw')
    download_url(districts_uri, raw_path)
    with raw_path.open('rt') as f:
        d = json.load(f)
        d['features'] = features = [simplify_feature(_) for _ in d['features']]
    with districts_path.open('wt') as f:
        json.dump(d, f)
else:
    with districts_path.open('rt') as f:
        d = json.load(f)
        features = d['features']
districts_collection = d
len(features)

In [ ]:
# ll $districts_path.parent -h

In [ ]:
# rm $districts_path

In [ ]:
import requests

def get_travel_packs(origin_strings, destination_strings):
    endpoint_uri = 'https://maps.googleapis.com/maps/api/distancematrix/json'
    origins_string = '|'.join(origin_strings)
    destinations_string = '|'.join(destination_strings)
    uri = f'{endpoint_uri}?origins={origins_string}&destinations={destinations_string}&key={GOOGLE_KEY}'
    response = requests.get(uri)
    d = response.json()
    origin_addresses = d['origin_addresses']
    travel_packs = []
    for origin_address, row in zip(origin_addresses, d['rows']):
        travel_packs.append((origin_address, row['elements'][0]['duration']['value']))
    return travel_packs

In [ ]:
def get_coordinate_string(coordinates):
    return ','.join(str(_) for _ in reversed(coordinates))

get_coordinate_string((0, 1))

In [ ]:
def get_random_point(geometry, count):
    pass

def get_sample_points(geometry, sample_size):
    # TODO: Use sample size
    return [geometry.centroid.coords[0]]

In [ ]:
def split(l, s):
    for i in range(0, len(l), s):
        yield l[i:i + s]

In [ ]:
import math

GOOGLE_DISTANCE_MATRIX_MAXIMUM_COUNT = 25
SAMPLE_SIZE = 1

destination_addresses = [destination_address]
origin_packs = []
for some_features in split(features, GOOGLE_DISTANCE_MATRIX_MAXIMUM_COUNT):
    origins = []
    for feature in some_features:
        x = feature['geometry']
        g = shape(x)
        origins.extend(get_sample_points(g, SAMPLE_SIZE))
    origin_strings = [get_coordinate_string(_) for _ in origins]
    destination_strings = destination_addresses
    travel_packs = get_travel_packs(origin_strings, destination_strings)
    for some_travel_packs, feature in zip(split(travel_packs, SAMPLE_COUNT), some_features):
        ts = []
        for origin_address, time_in_seconds in some_travel_packs:
            origin_packs.append((origin_address, time_in_seconds))
            ts.append(time_in_seconds)
        time_in_seconds = sum(ts) / len(ts)
        time_in_minutes = math.ceil(time_in_seconds / 60)
        feature['properties'] = {
            't': time_in_minutes,
        }

In [ ]:
with (output_folder / 'map.geojson').open('wt') as f:
    json.dump(districts_collection, f)

In [ ]:
ts = [_['properties']['t'] for _ in features]
len(ts)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ts, bins=10)
plt.title(f'Time to {destination_address} by {travel_name.title()}')
plt.xlabel(f'minutes')
plt.savefig(output_folder / 'histogram.png')